In [42]:
import datasets
import numpy as np
from collections import Counter
import tqdm

In [55]:
data = datasets.load_from_disk("lmsys_chat_englist_dataset")
#  lmsys_chat_englist_dataset  为lmsys/lmsys-chat-1m 过滤英文对话得到

In [56]:
data['train'][0]["conversation"][0]["content"]

'how can identity protection services help protect me against identity theft'

In [59]:
def filter_by_redacted(example, redacted=False):
    return example["redacted"] == redacted

# 过滤数据集，只保留 redacted 列值为 False 的数据。 代表对话未被编辑、修改过
filtered_redacted_dataset = data.filter(filter_by_redacted, fn_kwargs={"redacted": False})

In [60]:
filtered_redacted_dataset['train'].num_rows

511193

In [62]:
def filter_by_redacted(example, length=2048):
    return sum([ len(item["content"])  for item in example["conversation"]]) < length

# 过滤数据集，只保留 redacted 列值为 False 的数据。 代表对话未被编辑、修改过
filtered_len_dataset = filtered_redacted_dataset.filter(filter_by_redacted, fn_kwargs={"length": 2048})

In [67]:
filtered_len_dataset['train'].num_rows

387574

In [64]:
dialogue_lengths = []

for dialogue in tqdm.tqdm(filtered_len_dataset['train']):
    # print(dialogue)
    dialogue_lengths.append(len(dialogue['conversation']))

mean_dialogue_len = np.mean(dialogue_lengths)
print(f'Average dialogue length = {int(mean_dialogue_len)}')

100%|██████████| 387574/387574 [01:37<00:00, 3966.32it/s]

Average dialogue length = 2


In [65]:
# 使用Counter统计每个数字出现的次数
counts = Counter(dialogue_lengths)

In [66]:
counts

Counter({2: 329714,
         4: 38167,
         6: 11560,
         8: 4295,
         10: 1863,
         12: 842,
         14: 438,
         16: 271,
         18: 146,
         20: 88,
         22: 50,
         24: 32,
         26: 21,
         30: 21,
         34: 13,
         32: 13,
         28: 11,
         38: 5,
         40: 4,
         48: 4,
         42: 3,
         36: 3,
         50: 3,
         46: 2,
         58: 1,
         56: 1,
         52: 1,
         68: 1,
         54: 1})

In [68]:
filtered_len_dataset.save_to_disk("lmsys_chat_filter_dataset")
# 语言为英文，未被剪辑过，对话总字符长度 < 2048

Saving the dataset (0/3 shards):   0%|          | 0/387574 [00:00<?, ? examples/s]